In [1]:
# encoding=utf8  
import numpy as np
import tensorflow as tf

def unpickle(file):
  import _pickle as cPickle
  fo = open(file, 'rb')
  dict = cPickle.load(fo,encoding='latin1')
  fo.close()
  if 'data' in dict:
    dict['data'] = dict['data'].reshape((-1, 3, 32, 32)).swapaxes(1, 3).swapaxes(1, 2).reshape(-1, 32*32*3) / 256.

  return dict

In [2]:
def load_data_one(f):
  batch = unpickle(f)
  data = batch['data']
  labels = batch['labels']
  print ("Loading %s: %d" % (f, len(data)))
  return data, labels


In [3]:

def load_data(files, data_dir, label_count):
  data, labels = load_data_one(data_dir + '/' + files[0])
  for f in files[1:]:
    data_n, labels_n = load_data_one(data_dir + '/' + f)
    data = np.append(data, data_n, axis=0)
    labels = np.append(labels, labels_n, axis=0)
  labels = np.array([ [ float(i == label) for i in range(label_count) ] for label in labels ])
  return data, labels

In [4]:
def run_in_batch_avg(session, tensors, batch_placeholders, feed_dict={}, batch_size=200):                              
  res = [ 0 ] * len(tensors)                                                                                           
  batch_tensors = [ (placeholder, feed_dict[ placeholder ]) for placeholder in batch_placeholders ]                    
  total_size = len(batch_tensors[0][1])                                                                                
  batch_count = (total_size + batch_size - 1) / batch_size                                                             
  for batch_idx in range(batch_count):                                                                                
    current_batch_size = None                                                                                          
    for (placeholder, tensor) in batch_tensors:                                                                        
      batch_tensor = tensor[ batch_idx*batch_size : (batch_idx+1)*batch_size ]                                         
      current_batch_size = len(batch_tensor)                                                                           
      feed_dict[placeholder] = tensor[ batch_idx*batch_size : (batch_idx+1)*batch_size ]                               
    tmp = session.run(tensors, feed_dict=feed_dict)                                                                    
    res = [ r + t * current_batch_size for (r, t) in zip(res, tmp) ]                                                   
  return [ r / float(total_size) for r in res ]


In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial)

def conv2d(input, in_features, out_features, kernel_size, with_bias=False):
  W = weight_variable([ kernel_size, kernel_size, in_features, out_features ])
  conv = tf.nn.conv2d(input, W, [ 1, 1, 1, 1 ], padding='SAME')
  if with_bias:
    return conv + bias_variable([ out_features ])
  return conv

def batch_activ_conv(current, in_features, out_features, kernel_size, is_training, keep_prob):
  current = tf.contrib.layers.batch_norm(current, scale=True, is_training=is_training, updates_collections=None)
  current = tf.nn.relu(current)
  current = conv2d(current, in_features, out_features, kernel_size)
  current = tf.nn.dropout(current, keep_prob)
  return current

def block(input, layers, in_features, growth, is_training, keep_prob):
  current = input
  features = in_features
  for idx in range(layers):
    tmp = batch_activ_conv(current, features, growth, 3, is_training, keep_prob)
    current = tf.concat((current, tmp),3)
    features += growth
  return current, features

def avg_pool(input, s):
  return tf.nn.avg_pool(input, [ 1, s, s, 1 ], [1, s, s, 1 ], 'VALID')



In [6]:
data_dir = './data'
image_size = 32
image_dim = image_size * image_size * 3
# meta = unpickle(data_dir + '/batches.meta')
# label_names = meta['label_names']
# label_count = len(label_names)
label_count = 10
# train_files = [ 'data_batch_%d' % d for d in range(1, 6) ]
# train_data, train_labels = load_data(train_files, data_dir, label_count)
# pi = np.random.permutation(len(train_data))
# train_data, train_labels = train_data[pi], train_labels[pi]
# test_data, test_labels = load_data([ 'test_batch' ], data_dir, label_count)
# print ("Train:", np.shape(train_data), np.shape(train_labels))
# print ("Test:", np.shape(test_data), np.shape(test_labels))
# data = { 'train_data': train_data,
#   'train_labels': train_labels,
#   'test_data': test_data,
#   'test_labels': test_labels }
depth = 40

In [7]:

weight_decay = 1e-4
layers = int((depth - 4) / 3)
graph = tf.Graph()

xs = tf.placeholder("float", shape=[None, image_dim])
ys = tf.placeholder("float", shape=[None, label_count])
lr = tf.placeholder("float", shape=[])
keep_prob = tf.placeholder(tf.float32)
is_training = tf.placeholder("bool", shape=[])


current = tf.reshape(xs, [ -1, 32, 32, 3 ])
current = conv2d(current, 3, 16, 3)

current, features = block(current, layers, 16, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)

current = tf.contrib.layers.batch_norm(current, scale=True, is_training=is_training, updates_collections=None)
current = tf.nn.relu(current)
current = avg_pool(current, 8)
final_dim = features
current = tf.reshape(current, [ -1, final_dim ])
Wfc = weight_variable([ final_dim, label_count ])
bfc = bias_variable([ label_count ])
ys_ = tf.nn.softmax( tf.matmul(current, Wfc) + bfc )

cross_entropy = -tf.reduce_mean(ys * tf.log(ys_ + 1e-12))
l2 = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
train_step = tf.train.MomentumOptimizer(lr, 0.9, use_nesterov=True).minimize(cross_entropy + l2 * weight_decay)
correct_prediction = tf.equal(tf.argmax(ys_, 1), tf.argmax(ys, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    


In [8]:
para_dict={}
for k in tf.global_variables():
    if k not in tf.contrib.framework.get_variables_by_suffix('Momentum'):
            para_dict[k.name[:-2]] = k

In [9]:
sess=tf.InteractiveSession()
saver = tf.train.Saver(para_dict)
saver.restore(sess,'./inqmodel/stage2/inqcom1697100s.ckpt')
#saver.restore(sess,'./inqmodel/stage2/64pinq100/64pinq100.ckpt')
#saver.restore(sess,'./inqmodel/stage2/32inc100/inq32inc910.ckpt')

#saver.restore(sess,'./modellog/weightonlypara93.ckpt')

INFO:tensorflow:Restoring parameters from ./inqmodel/stage2/inqcom1697100s.ckpt


In [10]:
import config

In [11]:
save_dict ={}
for k in para_dict.items():
    weight = k[1].eval() 
    if k[0] in config.all_para:        
        
        weight[weight !=0] = (np.floor(np.log2(abs(weight[weight != 0]*4/3))))*np.sign(weight[weight != 0])
       # weight = weight.astype(int)
    save_dict[k[0]] = weight
save_dict

{'BatchNorm/beta': array([ 0.1005896 ,  0.20255108, -0.17592074, -0.13554171,  0.01152138,
         0.14950271,  0.06900312,  0.22282477, -0.02903798,  0.16441177,
        -0.16324764, -0.01700974,  0.28107855, -0.0472836 , -0.02780186,
         0.18496597], dtype=float32),
 'BatchNorm/gamma': array([ 0.12662669,  0.24184744,  0.23211329,  0.15013759,  0.13286395,
         0.11641019,  0.22883593,  0.00057689,  0.16092011,  0.13158932,
         0.24368289,  0.01502605,  0.18039015,  0.11582124,  0.14466676,
         0.20597854], dtype=float32),
 'BatchNorm/moving_mean': array([-0.00383548, -0.0118994 , -0.05140854,  0.0213616 ,  0.02971593,
         0.00891912,  0.00347926,  0.07220513, -0.03956654, -0.03991182,
        -0.2145973 , -0.3902587 , -0.01427782,  0.22812296, -0.01625541,
         0.04085527], dtype=float32),
 'BatchNorm/moving_variance': array([ 0.00940174,  0.0180527 ,  0.0406678 ,  0.13040502,  0.06178899,
         0.01516236,  0.01403405,  0.11757266,  0.08575732,  0.00

In [21]:
save_dict ={}
for k in para_dict.items():
    weight = k[1].eval() 
    if k[0] in config.all_para:        
        
        weight[weight !=0] = (np.floor(np.log2(abs(weight[weight != 0]*4/3))))#*np.sign(weight[weight != 0])
        weight = weight.astype(int)
    save_dict[k[0]] = weight
save_dict

{'BatchNorm/beta': array([ 0.08321584,  0.10316731, -0.12733406, -0.06267124,  0.00620966,
         0.1026805 ,  0.0900551 ,  0.14423433, -0.05722855,  0.09593607,
        -0.10592815, -0.01391049,  0.1799453 , -0.02109962, -0.026368  ,
         0.14144515], dtype=float32),
 'BatchNorm/gamma': array([ 0.1140104 ,  0.18321645,  0.15764038,  0.14057337,  0.07328121,
         0.09731749,  0.12755632,  0.00383015,  0.10769176,  0.07336008,
         0.14297469,  0.02200568,  0.13547945,  0.07076677,  0.10066565,
         0.18450475], dtype=float32),
 'BatchNorm/moving_mean': array([-0.00293689, -0.01852734,  0.00743793, -0.02549321, -0.01864818,
        -0.00859262, -0.01582259, -0.01386582, -0.02563288, -0.0271016 ,
        -0.18502724, -0.22533597, -0.01174836,  0.32815731, -0.023964  ,
        -0.00125048], dtype=float32),
 'BatchNorm/moving_variance': array([ 0.00271165,  0.01296419,  0.01755416,  0.04006873,  0.04356964,
         0.00626143,  0.00763565,  0.05565015,  0.03049036,  0.00

In [22]:
pruned_tol = np.zeros(128)

for k in save_dict.items():
    max_layer = -64
    min_layer = 64
    weight = k[1]
    for j in range(-64,64):
        pruned_tol[j+64] += len(weight[weight == j])
        if len(weight[weight == j]) > 0:
            if j > max_layer: max_layer = j
            if j < min_layer: min_layer = j
    if k[0] in config.all_para:
        print(k[0],'max:',max_layer,'min',min_layer,'diff',max_layer - min_layer,'j')
    #print (k[0],len(weight[weight ==0]))
pruned_tol.astype(int)

Variable max: -1 min -11 diff 10 j
Variable_1 max: -2 min -15 diff 13 j
Variable_2 max: -3 min -22 diff 19 j
Variable_3 max: -3 min -20 diff 17 j
Variable_4 max: -3 min -27 diff 24 j
Variable_5 max: -3 min -23 diff 20 j
Variable_6 max: -3 min -25 diff 22 j
Variable_7 max: -3 min -29 diff 26 j
Variable_8 max: -3 min -25 diff 22 j
Variable_9 max: -3 min -22 diff 19 j
Variable_10 max: -3 min -28 diff 25 j
Variable_11 max: -4 min -28 diff 24 j
Variable_12 max: -3 min -28 diff 25 j
Variable_13 max: -3 min -20 diff 17 j
Variable_14 max: -4 min -21 diff 17 j
Variable_15 max: -3 min -22 diff 19 j
Variable_16 max: -3 min -19 diff 16 j
Variable_17 max: -4 min -24 diff 20 j
Variable_18 max: -4 min -21 diff 17 j
Variable_19 max: -3 min -23 diff 20 j
Variable_20 max: -4 min -25 diff 21 j
Variable_21 max: -4 min -21 diff 17 j
Variable_22 max: -4 min -25 diff 21 j
Variable_23 max: -4 min -22 diff 18 j
Variable_24 max: -4 min -21 diff 17 j
Variable_25 max: -4 min -26 diff 22 j
Variable_26 max: -3 min 

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      1,
            0,      0,      0,      1,      8,     12,     25,     48,
          101,    202,    425,    768,   1468,   2483,   4431,   6701,
         9858,  14281,  19679,  25483,  33012,  46047,  68933, 107667,
       165185, 219531, 194014,  72804,   7256,    523,    186,     59,
          424,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [23]:
pruned_tol[64]

424.0

In [17]:
pruned_tol = np.zeros(64)
for k in save_dict.items():
    weight = k[1]
    for j in range(64):
        pruned_tol[j] += len(weight[abs(weight) == j])
    #print (k[0],len(weight[weight ==0]))
pruned_tol.astype(int)

array([   424,     59,    186,    523,   7256,  72804, 194014, 219531,
       165185, 107667,  68933,  46047,  33012,  25483,  19679,  14281,
         9858,   6701,   4431,   2483,   1468,    768,    425,    202,
          101,     48,     25,     12,      8,      1,      0,      0,
            0,      1,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

In [31]:
sess=tf.InteractiveSession()
saver = tf.train.Saver(para_dict)
#.restore(sess,'./inqmodel/stage2/64inc100/inq64inc100.ckpt')
saver.restore(sess,'./inqmodel/stage2/64pinq100/64pinq100.ckpt')
#saver.restore(sess,'./modellog/weightonlypara93.ckpt')

INFO:tensorflow:Restoring parameters from ./inqmodel/stage2/64pinq100/64pinq100.ckpt


In [18]:
pruned_tol = np.zeros(64)
for k in save_dict.items():
    weight = k[1]
    for j in range(64):
        pruned_tol[j] += len(weight[abs(weight) == j])
    #print (k[0],len(weight[weight ==0]))
pruned_tol.astype(int)

array([113959,     79,    313,   1961,  30451, 138162,  76286,  10362,
        80758, 173821, 132774,  96809,  62824,  43390,  34037,   5630,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

In [16]:
pruned_tol = 0
for k in save_dict.items():
    weight = k[1]
    pruned_tol += len(weight[weight ==0])
    print (k[0],len(weight[weight ==0]))
pruned_tol

Variable 0
BatchNorm/beta 0
BatchNorm/gamma 0
BatchNorm/moving_mean 0
BatchNorm/moving_variance 0
Variable_1 31
BatchNorm_1/beta 0
BatchNorm_1/gamma 0
BatchNorm_1/moving_mean 0
BatchNorm_1/moving_variance 0
Variable_2 578
BatchNorm_2/beta 0
BatchNorm_2/gamma 0
BatchNorm_2/moving_mean 0
BatchNorm_2/moving_variance 0
Variable_3 45
BatchNorm_3/beta 0
BatchNorm_3/gamma 0
BatchNorm_3/moving_mean 0
BatchNorm_3/moving_variance 0
Variable_4 1257
BatchNorm_4/beta 0
BatchNorm_4/gamma 0
BatchNorm_4/moving_mean 0
BatchNorm_4/moving_variance 0
Variable_5 608
BatchNorm_5/beta 0
BatchNorm_5/gamma 0
BatchNorm_5/moving_mean 0
BatchNorm_5/moving_variance 0
Variable_6 714
BatchNorm_6/beta 0
BatchNorm_6/gamma 0
BatchNorm_6/moving_mean 0
BatchNorm_6/moving_variance 0
Variable_7 746
BatchNorm_7/beta 0
BatchNorm_7/gamma 0
BatchNorm_7/moving_mean 0
BatchNorm_7/moving_variance 0
Variable_8 507
BatchNorm_8/beta 0
BatchNorm_8/gamma 0
BatchNorm_8/moving_mean 0
BatchNorm_8/moving_variance 0
Variable_9 297
BatchNor

113959

In [12]:
import pickle
# create dict
# save dict
f1 = open("C:/Users/lhlne/Desktop/project/densenet/compare/inq100.txt","wb")
pickle.dump(save_dict, f1)
f1.close()
# load dict
f2 = open("C:/Users/lhlne/Desktop/project/densenet/compare/inq100.txt","rb")
load_list = pickle.load(f2)
f2.close()
# print 
print(load_list)

{'Variable': array([[[[-1.,  0.,  3., -3.,  0.,  0.,  4., -2.,  0.,  0., -3.,  0.,  2.,
           0.,  4.,  3.],
         [ 0.,  3.,  0., -3.,  0.,  0.,  0., -2.,  0.,  0.,  0.,  0.,  0.,
           0.,  3., -3.],
         [ 1.,  0., -3., -2.,  0.,  0.,  3., -2.,  0.,  4.,  4.,  0., -2.,
           0.,  3.,  4.]],

        [[-2., -2.,  3.,  0.,  2., -3., -2., -2.,  4., -3.,  0., -3.,  1.,
           0.,  0.,  3.],
         [ 0.,  2.,  0.,  3.,  2.,  0., -2., -2.,  3.,  0.,  3.,  0.,  0.,
           0.,  0., -2.],
         [ 3.,  0., -3.,  3.,  3., -3., -1., -2.,  4.,  2.,  3.,  0., -1.,
           0.,  0.,  3.]],

        [[ 3., -4.,  4.,  0.,  4.,  3.,  0., -2., -4., -3.,  0., -4.,  1.,
           0.,  0.,  3.],
         [ 0.,  3., -3.,  3.,  0.,  0.,  0., -2., -3.,  0.,  3.,  0., -3.,
           0.,  0., -4.],
         [-3.,  0., -2.,  3.,  0.,  3.,  4., -3.,  0.,  2.,  3.,  0., -2.,
           0.,  0.,  0.]]],


       [[[-1., -3.,  3., -1.,  3., -3., -3.,  4.,  2.,  3., -4.,  0., 

In [16]:
f2 = open("C:/Users/lhlne/Desktop/project/densenet/inqmodel/stage1/inqfinal64inc20adj.txt","rb")
load_list = pickle.load(f2)
f2.close()
# print 
print(load_list)

{'Variable': array([[[[ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.],
         [ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.]],

        [[ 0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,
           1.,  1.,  1.],
         [ 1.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  0.],
         [ 0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,
           1.,  1.,  1.]],

        [[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
           1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,
           1.,  1.,  1.]]],


       [[[ 0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 

In [17]:
config.inq_para

{'Variable': 32,
 'Variable_1': 32,
 'Variable_10': 32,
 'Variable_11': 32,
 'Variable_12': 32,
 'Variable_13': 32,
 'Variable_14': 32,
 'Variable_15': 32,
 'Variable_16': 32,
 'Variable_17': 32,
 'Variable_18': 32,
 'Variable_19': 32,
 'Variable_2': 32,
 'Variable_20': 32,
 'Variable_21': 32,
 'Variable_22': 32,
 'Variable_23': 32,
 'Variable_24': 32,
 'Variable_25': 32,
 'Variable_26': 32,
 'Variable_27': 32,
 'Variable_28': 32,
 'Variable_29': 32,
 'Variable_3': 32,
 'Variable_30': 32,
 'Variable_31': 32,
 'Variable_32': 32,
 'Variable_33': 32,
 'Variable_34': 32,
 'Variable_35': 32,
 'Variable_36': 32,
 'Variable_37': 32,
 'Variable_38': 32,
 'Variable_39': 32,
 'Variable_4': 32,
 'Variable_5': 32,
 'Variable_6': 32,
 'Variable_7': 32,
 'Variable_8': 32,
 'Variable_9': 32}

In [16]:
para_dict['Variable_3'].eval()

array([[[[  0.00000000e+00,   0.00000000e+00,   3.47397669e-04, ...,
            0.00000000e+00,   3.58613906e-04,   8.46622512e-04],
         [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,  -7.80620612e-05,   1.56250000e-02],
         ..., 
         [ -1.56250000e-02,   1.56250000e-02,   0.00000000e+00, ...,
            1.56250000e-02,   1.56250000e-02,  -1.56250000e-02],
         [ -1.56250000e-02,   0.00000000e+00,  -1.56250000e-02, ...,
            0.00000000e+00,   0.00000000e+00,  -1.56250000e-02],
         [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,  -1.56250000e-02]],

        [[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [  0.00000000e+00,   0.00000000e+00,

In [17]:
from matplotlib import pyplot as plt
for i in para_dict.items():
    if 'Variable' in i[0]:
        a = para_dict[i[0]].eval()
        b = np.reshape(a,[-1])
        c = sum(b ==0)
        d = np.shape(b)
   #     e = np.percentile(b, 75)
        print(i[0])
 #       print(e)
   #     print(int(np.log10(abs(e))))
        print('percentage',c/d)
#         print(sum(b ==0))
#         plt.hist(abs(b)) 
#         plt.title(i[0]) 
#         plt.show()

Variable
percentage [ 0.]
Variable_1
percentage [ 0.]
Variable_2
percentage [ 0.]
Variable_3
percentage [ 0.]
Variable_4
percentage [ 0.]
Variable_5
percentage [ 0.]
Variable_6
percentage [ 0.]
Variable_7
percentage [ 0.]
Variable_8
percentage [ 0.]
Variable_9
percentage [ 0.]
Variable_10
percentage [ 0.]
Variable_11
percentage [ 0.]
Variable_12
percentage [ 0.]
Variable_13
percentage [ 0.]
Variable_14
percentage [ 0.]
Variable_15
percentage [ 0.]
Variable_16
percentage [ 0.]
Variable_17
percentage [ 0.]
Variable_18
percentage [ 0.]
Variable_19
percentage [ 0.]
Variable_20
percentage [ 0.]
Variable_21
percentage [ 0.]
Variable_22
percentage [ 0.]
Variable_23
percentage [ 0.]
Variable_24
percentage [ 0.]
Variable_25
percentage [ 0.]
Variable_26
percentage [ 0.]
Variable_27
percentage [ 0.]
Variable_28
percentage [ 0.]
Variable_29
percentage [ 0.]
Variable_30
percentage [ 0.]
Variable_31
percentage [ 0.]
Variable_32
percentage [ 0.]
Variable_33
percentage [ 0.]
Variable_34
percentage [ 0

In [30]:
a = para_dict['Variable_27'].eval()
b = np.reshape(a,[-1])
print(sum(b <=0.00001))
print(np.shape(b))


25166
(32832,)


In [27]:
parm_dict={}
for k in tf.trainable_variables():
    parm_dict[k.name] = k
with tf.Session(graph=graph) as session:
    saver= tf.train.Saver(parm_dict) 
    saver.restore(session,"weights_densenet")

Variable:0
BatchNorm/beta:0
BatchNorm/gamma:0
Variable_1:0
BatchNorm_1/beta:0
BatchNorm_1/gamma:0
Variable_2:0
BatchNorm_2/beta:0
BatchNorm_2/gamma:0
Variable_3:0
BatchNorm_3/beta:0
BatchNorm_3/gamma:0
Variable_4:0
BatchNorm_4/beta:0
BatchNorm_4/gamma:0
Variable_5:0
BatchNorm_5/beta:0
BatchNorm_5/gamma:0
Variable_6:0
BatchNorm_6/beta:0
BatchNorm_6/gamma:0
Variable_7:0
BatchNorm_7/beta:0
BatchNorm_7/gamma:0
Variable_8:0
BatchNorm_8/beta:0
BatchNorm_8/gamma:0
Variable_9:0
BatchNorm_9/beta:0
BatchNorm_9/gamma:0
Variable_10:0
BatchNorm_10/beta:0
BatchNorm_10/gamma:0
Variable_11:0
BatchNorm_11/beta:0
BatchNorm_11/gamma:0
Variable_12:0
BatchNorm_12/beta:0
BatchNorm_12/gamma:0
Variable_13:0
BatchNorm_13/beta:0
BatchNorm_13/gamma:0
Variable_14:0
BatchNorm_14/beta:0
BatchNorm_14/gamma:0
Variable_15:0
BatchNorm_15/beta:0
BatchNorm_15/gamma:0
Variable_16:0
BatchNorm_16/beta:0
BatchNorm_16/gamma:0
Variable_17:0
BatchNorm_17/beta:0
BatchNorm_17/gamma:0
Variable_18:0
BatchNorm_18/beta:0
BatchNorm_18

In [7]:
depth = 40
weight_decay = 1e-4
layers = int((depth - 4) / 3)

xs = tf.placeholder("float", shape=[None, image_dim])
ys = tf.placeholder("float", shape=[None, label_count])
lr = tf.placeholder("float", shape=[])
keep_prob = tf.placeholder(tf.float32)
is_training = tf.placeholder("bool", shape=[])


current = tf.reshape(xs, [ -1, 32, 32, 3 ])
current = conv2d(current, 3, 16, 3)

current, features = block(current, layers, 16, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)
current = batch_activ_conv(current, features, features, 1, is_training, keep_prob)
current = avg_pool(current, 2)
current, features = block(current, layers, features, 12, is_training, keep_prob)

current = tf.contrib.layers.batch_norm(current, scale=True, is_training=is_training, updates_collections=None)
current = tf.nn.relu(current)
current = avg_pool(current, 8)
final_dim = features
current = tf.reshape(current, [ -1, final_dim ])
Wfc = weight_variable([ final_dim, label_count ])
bfc = bias_variable([ label_count ])
ys_ = tf.nn.softmax( tf.matmul(current, Wfc) + bfc )

cross_entropy = -tf.reduce_mean(ys * tf.log(ys_ + 1e-12))
l2 = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
train_step = tf.train.MomentumOptimizer(lr, 0.9, use_nesterov=True).minimize(cross_entropy + l2 * weight_decay)
correct_prediction = tf.equal(tf.argmax(ys_, 1), tf.argmax(ys, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.Session()
batch_size = 64
learning_rate = 0.1
tf.global_variables_initializer()
    # saver = tf.train.Saver()
    # train_data, train_labels = data['train_data'], data['train_labels']
    # batch_count = int(len(train_data) / batch_size)
    # batches_data = np.split(train_data[:batch_count * batch_size], batch_count)
    # batches_labels = np.split(train_labels[:batch_count * batch_size], batch_count)
    # print ("Batch per epoch: ", batch_count)
    # for epoch in range(1, 1+300):
    #   if epoch == 150: learning_rate = 0.01
    #   if epoch == 225: learning_rate = 0.001
    #   for batch_idx in range(batch_count):
    #     xs_, ys_ = batches_data[batch_idx], batches_labels[batch_idx]
    #     batch_res = session.run([ train_step, cross_entropy, accuracy ],
    #       feed_dict = { xs: xs_, ys: ys_, lr: learning_rate, is_training: True, keep_prob: 0.8 })
    #     if batch_idx % 100 == 0: print (epoch, batch_idx, batch_res[1:])

    #   save_path = saver.save(session, 'densenet_%d.ckpt' % epoch)
    #   test_results = run_in_batch_avg(session, [ cross_entropy, accuracy ], [ xs, ys ],
    #       feed_dict = { xs: data['test_data'], ys: data['test_labels'], is_training: False, keep_prob: 1. })
    #   print (epoch, batch_res[1:], test_results)


<tf.Operation 'init' type=NoOp>